In [ ]:
# Experimental Topology
# lockin AO 3 -> Analog Attenuator Vc (Control Voltages)
# Keithley Output -> Voltage divider with 1 MOhm, DAQmx AI1
# PSPL Trigger -> DAQmx PFI0

In [1]:
cd ..

C:\Users\qlab\Documents\GitHub\pycontrol


In [2]:
from pycontrol.instruments.keysight import *
from pycontrol.instruments.picosecond import Picosecond10070A
from pycontrol.instruments.stanford import SR865
from pycontrol.instruments.keithley import Keithley2400
from pycontrol.instruments.ami import AMI430

from PyDAQmx import *

from pycontrol.experiment import FloatParameter, IntParameter, Experiment
from pycontrol.stream import DataStream, DataAxis, DataStreamDescriptor, OutputConnector
from pycontrol.filters.debug import Print
from pycontrol.filters.io import WriteToHDF5, ProgressBar

import asyncio
import numpy as np
import time
import matplotlib.pyplot as plt
from analysis.h5shell import h5shell

In [3]:
from experiments.Pulse_Switching_Search_Reset import ResetSearchExperiment

In [4]:
exp = ResetSearchExperiment()
exp.sample = "CSHE2-C4R2"
exp.field.value = 0.0133
exp.duration = 10e-9
exp.measure_current = 3e-6
amps = np.arange(-0.2, 0.21, 0.05)
amps = np.append(amps, np.flipud(amps))
exp.amplitudes = amps
exp.init_streams()
exp.add_sweep(exp.reps_over, np.linspace(0,9,2))

<SweptParameter: reps_over>

In [5]:
wr = WriteToHDF5("data\CSHE-Switching\CSHE-Die2-C4R2\CSHE2-C4R2-Search_Reset_2016-06-29.h5")
pbar = ProgressBar(num=1, notebook=True)
edges = [(exp.daq_buffer, wr.data), (exp.daq_buffer, pbar.data)]
# edges = [(exp.daq_buffer, wr.data)]
exp.set_graph(edges)
exp.init_instruments()

In [6]:
exp.run_sweeps()
exp.shutdown_instruments()

Prepare to read buffer.
Done reading buffer: 92160 points
Prepare to read buffer.
Done reading buffer: 92160 points
Warning failed to stop task.


In [7]:
f = h5shell(wr.filename,'r')
dset= f[f.grep('data')[-1]]
buffers = dset.value
f.close()
# Plot the result
NUM = len(amps)
buff_mean = np.mean(buffers, axis=(2,3))
mean_state = np.mean(buff_mean, axis=0)

In [8]:
fig = plt.figure()
for i in range(NUM):
    plt.plot(amps[i]*np.ones(buff_mean[:,i].size),
            1e-3*buff_mean[:,i]/max(exp.measure_current,1e-7),
                '.', color='blue')
plt.plot(amps, 1e-3*mean_state/max(exp.measure_current,1e-7), '-', color='red')
plt.xlabel("AWG amplitude (V)", size=14);
plt.ylabel("Resistance (kOhm)", size=14);
plt.title("AWG Reset Amplitude Search")
plt.show()